In [ ]:
### All code I write here are in GPL - Feel free to reuse it

In [1]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv, find_dotenv
load_dotenv()
AZURE_OPENAI_API_KEY = os.getenv('AZURE_OPENAI_API_KEY')
END_POINT = os.getenv("AZURE_OPENAI_ENDPOINT")
OPENAI_API_VERSION = os.getenv("OPENAI_API_VERSION")
DEPLOYMENT_NAME = os.getenv("DEPLOYMENT_NAME")



In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
from llama_index.core.tools import FunctionTool

def add(x: int, y: int) -> int:
    """Adds two integers together."""
    return x + y

def mystery(x: int, y: int) -> int: 
    """Mystery function that operates on top of two numbers."""
    return (x + y) * (x + y)


add_tool = FunctionTool.from_defaults(fn=add)
mystery_tool = FunctionTool.from_defaults(fn=mystery)

In [5]:
from llama_index.core import Settings
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding

Settings.llm = AzureOpenAI( model="gpt-4-32k",
    deployment_name=DEPLOYMENT_NAME,
    api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=END_POINT,
    api_version=OPENAI_API_VERSION,)
Settings.embed_model = AzureOpenAIEmbedding(    model="text-embedding-ada-002",
    deployment_name="text-embedding-default",
    api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=END_POINT,
    api_version=OPENAI_API_VERSION,)

In [25]:
llm = AzureOpenAI( model="gpt-4-32k",
    deployment_name=DEPLOYMENT_NAME,
    api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=END_POINT,
    api_version=OPENAI_API_VERSION,)

response = llm.predict_and_call(
    [add_tool, mystery_tool], 
    "Tell me the output of the mystery function on 2 and 9", 
    verbose=True
)
print(str(response))

=== Calling Function ===
Calling function: mystery with args: {"x": 2, "y": 9}
=== Function Output ===
121
121


In [11]:
response = llm.predict_and_call(
    [add_tool, mystery_tool], 
    "I have 2 mangoes and she have 3. Homuch we have in total?", 
    verbose=True
)

=== Calling Function ===
Calling function: add with args: {"x": 2, "y": 3}
=== Function Output ===
5


## Define an Auto-Retrieval Tool

In [8]:
from llama_index.core import SimpleDirectoryReader
# load documents
documents = SimpleDirectoryReader(input_files=["metagpt.pdf"]).load_data()

In [9]:
from llama_index.core.node_parser import SentenceSplitter
splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

In [10]:
print(nodes[0].get_content(metadata_mode="all"))

page_label: 1
file_name: metagpt.pdf
file_path: metagpt.pdf
file_type: application/pdf
file_size: 16715764
creation_date: 2024-08-27
last_modified_date: 2024-08-27

Preprint
METAGPT: M ETA PROGRAMMING FOR A
MULTI -AGENT COLLABORATIVE FRAMEWORK
Sirui Hong1∗, Mingchen Zhuge2∗, Jonathan Chen1, Xiawu Zheng3, Yuheng Cheng4,
Ceyao Zhang4,Jinlin Wang1,Zili Wang ,Steven Ka Shing Yau5,Zijuan Lin4,
Liyang Zhou6,Chenyu Ran1,Lingfeng Xiao1,7,Chenglin Wu1†,J¨urgen Schmidhuber2,8
1DeepWisdom,2AI Initiative, King Abdullah University of Science and Technology,
3Xiamen University,4The Chinese University of Hong Kong, Shenzhen,
5Nanjing University,6University of Pennsylvania,
7University of California, Berkeley,8The Swiss AI Lab IDSIA/USI/SUPSI
ABSTRACT
Remarkable progress has been made on automated problem solving through so-
cieties of agents based on large language models (LLMs). Existing LLM-based
multi-agent systems can already solve simple dialogue tasks. Solutions to more
complex tasks, however, 

In [12]:
from llama_index.core import VectorStoreIndex

vector_index = VectorStoreIndex(nodes)
query_engine = vector_index.as_query_engine(similarity_top_k=2)

In [20]:
from llama_index.core.vector_stores import MetadataFilters

query_engine = vector_index.as_query_engine(
    similarity_top_k=2,
    filters=MetadataFilters.from_dicts(
        [
            {"key": "page_label", "value": "2"} ##Limiting results only from page 2
        ]
    ),
    verbose=True
)

response = query_engine.query(
    "What are some high-level results of MetaGPT?", 
)

In [21]:
print(str(response))

MetaGPT has achieved notable results in code generation benchmarks, setting a new state-of-the-art with 85.9% and 87.7% in Pass@1. Compared to other popular frameworks for creating complex software projects, MetaGPT excels in handling higher levels of software complexity and offering extensive functionality. In experimental evaluations, MetaGPT has achieved a 100% task completion rate, demonstrating its robustness and efficiency.


In [19]:
for n in response.source_nodes:
    print(n.metadata)

{'page_label': '2', 'file_name': 'metagpt.pdf', 'file_path': 'metagpt.pdf', 'file_type': 'application/pdf', 'file_size': 16715764, 'creation_date': '2024-08-27', 'last_modified_date': '2024-08-27'}


#### Auto-Retieval - Filter using metadata

In [23]:
from typing import List
from llama_index.core.vector_stores import FilterCondition


def vector_query(
    query: str, 
    page_numbers: List[str]
) -> str:
    """Perform a vector search over an index.
    
    query (str): the string query to be embedded.
    page_numbers (List[str]): Filter by set of pages. Leave BLANK if we want to perform a vector search
        over all pages. Otherwise, filter by the set of specified pages.
    
    """

    metadata_dicts = [
        {"key": "page_label", "value": p} for p in page_numbers
    ]
    
    query_engine = vector_index.as_query_engine(
        similarity_top_k=2,
        filters=MetadataFilters.from_dicts(
            metadata_dicts,
            condition=FilterCondition.OR
        )
    )
    response = query_engine.query(query)
    return response
    

vector_query_tool = FunctionTool.from_defaults(
    name="vector_tool",
    fn=vector_query
)

In [26]:
response = llm.predict_and_call(
    [vector_query_tool], 
    "What are the high-level results of MetaGPT as described on page 2?", 
    verbose=True
)

=== Calling Function ===
Calling function: vector_tool with args: {"query": "high-level results of MetaGPT", "page_numbers": ["2"]}
=== Function Output ===
MetaGPT has achieved notable results in the field of meta-programming. In code generation benchmarks, it has set a new state-of-the-art with 85.9% and 87.7% in Pass@1. Compared to other popular frameworks for creating complex software projects, MetaGPT stands out in handling higher levels of software complexity and offering extensive functionality. In experimental evaluations, it has achieved a 100% task completion rate, demonstrating its robustness and efficiency.


In [27]:
response = llm.predict_and_call(
    [vector_query_tool], 
    "Compare contents in pages 2 and 3?", 
    verbose=True
)
##In this case the two pages are filtered out for

=== Calling Function ===
Calling function: vector_tool with args: {"query": "compare contents", "page_numbers": ["2", "3"]}
=== Function Output ===
The contents of the two pages primarily revolve around the introduction and explanation of MetaGPT, a meta-programming framework for LLM-based multi-agent systems. The first page discusses the Natural Language-Based Society of Mind (NLSOM) and the challenges of multi-agent cooperation. It introduces MetaGPT as a solution to these challenges, emphasizing the application of Standard Operating Procedures (SOPs) in multi-agent frameworks.

The second page delves deeper into the application of SOPs in MetaGPT, explaining how they promote collaboration among various roles and increase the success rate of target code generation. It also highlights the unique features of MetaGPT, such as its ability to decompose complex tasks into specific procedures assigned to various roles. The page further discusses how MetaGPT stands out from other projects li

In [28]:
for n in response.source_nodes:
    print(n.metadata)

{'page_label': '3', 'file_name': 'metagpt.pdf', 'file_path': 'metagpt.pdf', 'file_type': 'application/pdf', 'file_size': 16715764, 'creation_date': '2024-08-27', 'last_modified_date': '2024-08-27'}
{'page_label': '2', 'file_name': 'metagpt.pdf', 'file_path': 'metagpt.pdf', 'file_type': 'application/pdf', 'file_size': 16715764, 'creation_date': '2024-08-27', 'last_modified_date': '2024-08-27'}


#### Add more tools!

In [29]:
from llama_index.core import SummaryIndex
from llama_index.core.tools import QueryEngineTool

summary_index = SummaryIndex(nodes)
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
summary_tool = QueryEngineTool.from_defaults(
    name="summary_tool",
    query_engine=summary_query_engine,
    description=(
        "Useful if you want to get a summary of MetaGPT"
    ),
)

In [30]:
response = llm.predict_and_call(
    [vector_query_tool, summary_tool], 
    "What are the MetaGPT comparisons with ChatDev described on page 8?", 
    verbose=True
)
##Still it calls vector tool with page no 8 (metadata filer)

=== Calling Function ===
Calling function: vector_tool with args: {"query": "MetaGPT comparisons with ChatDev", "page_numbers": ["8"]}
=== Function Output ===
MetaGPT has been compared with ChatDev on several statistical indices. In terms of executability, MetaGPT scored higher with a score of 3.75 compared to ChatDev's 2.25. The running times for MetaGPT were shorter at 541 seconds, while ChatDev took 762 seconds. MetaGPT also used more tokens, with a count of 31,255 compared to ChatDev's 19,292. In terms of code statistics, MetaGPT generated more code files, had more lines of code per file, and had more total code lines than ChatDev. However, ChatDev had a higher productivity score of 248.9 compared to MetaGPT's 124.3. The human revision cost was lower for MetaGPT at 0.83, while it was 2.5 for ChatDev.


In [31]:
for n in response.source_nodes:
    print(n.metadata)

{'page_label': '8', 'file_name': 'metagpt.pdf', 'file_path': 'metagpt.pdf', 'file_type': 'application/pdf', 'file_size': 16715764, 'creation_date': '2024-08-27', 'last_modified_date': '2024-08-27'}


In [33]:
response = llm.predict_and_call(
    [vector_query_tool, summary_tool], 
    "What is a summary of the paper?", 
    verbose=True
)
#Now calls summery tool

=== Calling Function ===
Calling function: summary_tool with args: {"input": "MetaGPT: Towards Improving Large Language Models by Meta Learning"}
=== Function Output ===
MetaGPT is a meta-programming framework that enhances the problem-solving capabilities of multi-agent systems based on Large Language Models (LLMs). It models a group of agents as a simulated software company, with each agent having a specific role and expertise. MetaGPT incorporates Standard Operating Procedures (SOPs) into its framework, allowing for efficient task decomposition and coordination. It also introduces an executable feedback mechanism to improve code generation quality during runtime. In extensive experiments, MetaGPT achieves state-of-the-art performance on multiple benchmarks.
